In [1]:
#simple-tf-multiclass-classifieripynb
#
#by Joe Hahn
#jmh.datasciences@gmail.com
#27 January 2018

In [2]:
#this was adapted from https://machinelearningmastery.com/multi-class-classification-tutorial-keras-deep-learning-library/
import time
t_start = time.time()

In [3]:
#get the iris data
import os
os.system('rm -rf iris.data')
os.system('wget archive.ics.uci.edu/ml/machine-learning-databases/iris/iris.data')

0

In [4]:
#imports
import numpy
import pandas
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
from keras.utils import np_utils
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import LabelEncoder
from sklearn.pipeline import Pipeline

Using TensorFlow backend.


In [5]:
#initialize random number
seed = 7
numpy.random.seed(seed)

In [6]:
#load data
df = pandas.read_csv("iris.data", header=None)
dataset = df.values
X = dataset[:,0:4].astype(float)
Y = dataset[:,4]
df.head()

,0,1,2,3,4
0,5.1,3.5,1.4,0.2,Iris-setosa
1,4.9,3.0,1.4,0.2,Iris-setosa
2,4.7,3.2,1.3,0.2,Iris-setosa
3,4.6,3.1,1.5,0.2,Iris-setosa
4,5.0,3.6,1.4,0.2,Iris-setosa


In [7]:
#encode class values as integers
encoder = LabelEncoder()
encoder.fit(Y)
encoded_Y = encoder.transform(Y)
# convert integers to dummy variables (i.e. one hot encoded)
dummy_y = np_utils.to_categorical(encoded_Y)
N_inputs = X.shape[1]
N_outputs = dummy_y.shape[1]
print X.shape
print dummy_y.shape
print 'N_inputs = ', N_inputs
print 'N_outputs = ', N_outputs
print X[0:3]
print dummy_y[0:3]

(150, 4)
(150, 3)
N_inputs =  4
N_outputs =  3
[[ 5.1  3.5  1.4  0.2]
 [ 4.9  3.   1.4  0.2]
 [ 4.7  3.2  1.3  0.2]]
[[ 1.  0.  0.]
 [ 1.  0.  0.]
 [ 1.  0.  0.]]


In [8]:
#define model
def baseline_model():
    model = Sequential()
    model.add(Dense(8, input_dim=N_inputs, activation='relu'))
    model.add(Dense(N_outputs, activation='softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

In [9]:
#estimator
estimator = KerasClassifier(build_fn=baseline_model, epochs=200, batch_size=5, verbose=0)

In [10]:
#kfold cross validation
kfold = KFold(n_splits=10, shuffle=True, random_state=seed)

In [11]:
#evaluate model fit...this executes in 4 minutes on g2.2xl GPU instance...and 1.3 minutes on laptop no gpu!!!
results = cross_val_score(estimator, X, dummy_y, cv=kfold)
print("Baseline: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))

Baseline: 97.33% (4.42%)


In [13]:
#done!
t_finish = time.time()
delta_time_sec = t_finish - t_start
print 'execution time (minutes) = ', delta_time_sec/60.0

execution time (minutes) =  3.19440493186


In [14]:
#alternate to the above, with no kfold and estimator
model = Sequential()
model.add(Dense(8, input_dim=N_inputs, activation='relu'))
model.add(Dense(N_outputs, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])    
loss_history = model.fit(X, dummy_y, epochs=200, batch_size=5, verbose=0)
y_pred = model.predict(X)
y_pred[0:3]

array([[  9.99855042e-01,   1.44885722e-04,   2.30285561e-12],
       [  9.99248445e-01,   7.51467014e-04,   5.34652426e-11],
       [  9.99711215e-01,   2.88813986e-04,   1.61040469e-11]], dtype=float32)